In [2]:
import pandas as pd

from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline
from transformers import DistilBertForSequenceClassification

import torch
import tensorflow as tf
import pandas as pd
import json
#import gc

import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
#import statsmodels.api as sm
#import statsmodels.formula.api as smf
#%matplotlib inline
from sklearn.preprocessing import LabelEncoder

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

#import re
#import nltk
#from nltk.corpus import stopwords
#nltk.download('stopwords')
#stopw = stopwords.words('english')

import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot

import mpl_toolkits.mplot3d
import numpy as np
from sklearn.cluster import KMeans

from tqdm.auto import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


c:\Users\drodm\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\drodm\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\drodm\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\drodm\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is 

***Ao testar esta primeira versão, usar apenas atributos referentes ao tipo criatura. Generalizarei para as demais cartas nas próximas versões***

# Criando o dataframe, categorizando e dividindo as amostras por tipo de carta

In [2]:
prices = pd.read_csv('price.csv')

prices


,cardFinish,currency,date,gameAvailability,price,priceProvider,providerListing,uuid
0,normal,USD,2023-11-05,mtgo,0.08,cardhoarder,retail,f182e364-0439-5594-a6e6-75f7889ccf45
1,normal,USD,2023-11-05,mtgo,0.27,cardhoarder,retail,330deaa3-dd7a-52a8-bfbc-b323cd16a409
2,normal,USD,2023-11-05,mtgo,0.02,cardhoarder,retail,79e36956-b91f-580f-8309-7d9585a67560
3,normal,USD,2023-11-05,mtgo,0.17,cardhoarder,retail,6afb2b4c-530a-57d5-8e7f-871239f6fa05
4,normal,USD,2023-11-05,mtgo,0.02,cardhoarder,retail,b1fc2762-92aa-5a14-8509-a59cb611e376
...,...,...,...,...,...,...,...,...
47367,foil,USD,2023-11-05,mtgo,0.02,cardhoarder,retail,c799edb4-3a32-5976-be07-db94bc285cc0
47368,foil,USD,2023-11-05,paper,0.05,tcgplayer,buylist,c799edb4-3a32-5976-be07-db94bc285cc0
47369,normal,USD,2023-11-05,paper,0.17,tcgplayer,retail,c799edb4-3a32-5976-be07-db94bc285cc0
47370,foil,USD,2023-11-05,paper,0.98,tcgplayer,retail,c799edb4-3a32-5976-be07-db94bc285cc0


In [3]:
cards = pd.read_csv('cards.csv')

cards

C:\Users\drodm\AppData\Local\Temp\ipykernel_9832\2433561530.py:1: DtypeWarning:

Columns (3,7,12,16,20,23,25,26,30,31,32,33,35,39,40,41,47,52,53,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.



,artist,artistIds,asciiName,attractionLights,availability,boosterTypes,borderColor,cardParts,colorIdentity,colorIndicator,...,subsets,subtypes,supertypes,text,toughness,type,types,uuid,variations,watermark
0,Pete Venters,d54c4a1a-c0c5-4834-84db-125d341f3ad8,NaN,NaN,"mtgo, paper",default,black,NaN,W,NaN,...,NaN,"Human, Cleric",NaN,First strike (This creature deals combat damag...,4,Creature — Human Cleric,Creature,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,b7c19924-b4bf-56fc-aa73-f586e940bd42,NaN
1,Pete Venters,d54c4a1a-c0c5-4834-84db-125d341f3ad8,NaN,NaN,"mtgo, paper",default,black,NaN,W,NaN,...,NaN,"Human, Cleric",NaN,First strike (This creature deals combat damag...,4,Creature — Human Cleric,Creature,b7c19924-b4bf-56fc-aa73-f586e940bd42,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,NaN
2,Volkan Baǵa,93bec3c0-0260-4d31-8064-5d01efb4153f,NaN,NaN,"mtgo, paper",default,black,NaN,W,NaN,...,NaN,Angel,NaN,Flying\nWhen Angel of Mercy enters the battlef...,3,Creature — Angel,Creature,57aaebc1-850c-503d-9f6e-bb8d00d8bf7c,8fd4e2eb-3eb4-50ea-856b-ef638fa47f8a,NaN
3,Volkan Baǵa,93bec3c0-0260-4d31-8064-5d01efb4153f,NaN,NaN,"mtgo, paper",default,black,NaN,W,NaN,...,NaN,Angel,NaN,Flying\nWhen Angel of Mercy enters the battlef...,3,Creature — Angel,Creature,8fd4e2eb-3eb4-50ea-856b-ef638fa47f8a,57aaebc1-850c-503d-9f6e-bb8d00d8bf7c,NaN
4,Mark Zug,48e2b98c-5467-4671-bd42-4c3746115117,NaN,NaN,"mtgo, paper",default,black,NaN,W,NaN,...,NaN,NaN,NaN,Target creature gets +3/+3 and gains flying un...,NaN,Sorcery,Sorcery,55bd38ca-dc73-5c06-8f80-a6ddd2f44382,c5655330-5131-5f40-9d3e-0549d88c6e9e,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87213,Campbell White,d281eab4-463a-4ba8-9039-8943737960a0,NaN,NaN,"mtgo, paper",NaN,black,NaN,U,NaN,...,NaN,NaN,NaN,Kicker {1}{U} (You may pay an additional {1}{U...,NaN,Instant,Instant,3f492516-7767-5ed7-a1d4-e3f7c06aee2f,dd604910-0e81-5d56-b022-e08f21de0879,planeswalker
87214,Jason Rainville,6ed7e669-579b-443d-b223-e5cbcb2a7483,NaN,NaN,"mtgo, paper",NaN,black,NaN,B,NaN,...,NaN,NaN,NaN,Kicker {2}{B} (You may pay an additional {2}{B...,NaN,Sorcery,Sorcery,3f9a0369-5fe7-5aee-85fe-3cfaacd275af,d947d9cd-f855-5496-b5de-88006b49865f,planeswalker
87215,Campbell White,d281eab4-463a-4ba8-9039-8943737960a0,NaN,NaN,"mtgo, paper",NaN,black,NaN,R,NaN,...,NaN,NaN,NaN,Kicker {5} (You may pay an additional {5} as y...,NaN,Sorcery,Sorcery,97577e9e-69a9-5a8b-9c24-a72703790046,dbd65728-bba8-536e-a908-5dfa56068dcb,planeswalker
87216,Jonas De Ro,561ebf9e-8d93-4b57-8156-8826d0c19601,NaN,NaN,"mtgo, paper",NaN,black,NaN,G,NaN,...,NaN,NaN,NaN,Sacrifice a land. Search your library for up t...,NaN,Instant,Instant,deb51cbd-b890-5b2d-9d6f-7b896e16c6fd,7ef6cffb-335a-5831-8319-20b6a020f1c4,planeswalker


In [4]:
df = cards.merge(prices, left_on= 'uuid', right_on= 'uuid')

df

,artist,artistIds,asciiName,attractionLights,availability,boosterTypes,borderColor,cardParts,colorIdentity,colorIndicator,...,uuid,variations,watermark,cardFinish,currency,date,gameAvailability,price,priceProvider,providerListing
0,Marc Fishman,d50dd2c3-34ae-4eb1-8948-cb4b38d7e4b9,NaN,NaN,"mtgo, paper",default,white,NaN,W,NaN,...,cfe05eff-7428-53ef-afc8-788ef5964925,ab55424f-58d5-5d9b-bc0d-01e9368aa5f1,NaN,normal,USD,2023-11-05,mtgo,0.09,cardhoarder,retail
1,Marc Fishman,d50dd2c3-34ae-4eb1-8948-cb4b38d7e4b9,NaN,NaN,"mtgo, paper",default,white,NaN,W,NaN,...,cfe05eff-7428-53ef-afc8-788ef5964925,ab55424f-58d5-5d9b-bc0d-01e9368aa5f1,NaN,foil,USD,2023-11-05,mtgo,0.01,cardhoarder,retail
2,Marc Fishman,d50dd2c3-34ae-4eb1-8948-cb4b38d7e4b9,NaN,NaN,"mtgo, paper",default,white,NaN,W,NaN,...,cfe05eff-7428-53ef-afc8-788ef5964925,ab55424f-58d5-5d9b-bc0d-01e9368aa5f1,NaN,normal,USD,2023-11-05,paper,0.02,tcgplayer,buylist
3,Marc Fishman,d50dd2c3-34ae-4eb1-8948-cb4b38d7e4b9,NaN,NaN,"mtgo, paper",default,white,NaN,W,NaN,...,cfe05eff-7428-53ef-afc8-788ef5964925,ab55424f-58d5-5d9b-bc0d-01e9368aa5f1,NaN,foil,USD,2023-11-05,paper,4.15,tcgplayer,buylist
4,Marc Fishman,d50dd2c3-34ae-4eb1-8948-cb4b38d7e4b9,NaN,NaN,"mtgo, paper",default,white,NaN,W,NaN,...,cfe05eff-7428-53ef-afc8-788ef5964925,ab55424f-58d5-5d9b-bc0d-01e9368aa5f1,NaN,normal,USD,2023-11-05,paper,0.12,tcgplayer,retail
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47366,John Avon,798f3932-30e0-4420-aa3f-db4d613f89ca,NaN,NaN,"mtgo, paper",default,black,NaN,NaN,NaN,...,675ce267-e99d-523e-8722-402debaa9717,NaN,NaN,normal,USD,2023-11-05,paper,18.20,tcgplayer,retail
47367,John Avon,798f3932-30e0-4420-aa3f-db4d613f89ca,NaN,NaN,"mtgo, paper",default,black,NaN,NaN,NaN,...,675ce267-e99d-523e-8722-402debaa9717,NaN,NaN,normal,EUR,2023-11-05,paper,14.07,cardmarket,retail
47368,John Avon,798f3932-30e0-4420-aa3f-db4d613f89ca,NaN,NaN,"mtgo, paper",default,black,NaN,NaN,NaN,...,675ce267-e99d-523e-8722-402debaa9717,NaN,NaN,normal,USD,2023-11-05,paper,11.00,cardkingdom,buylist
47369,John Avon,798f3932-30e0-4420-aa3f-db4d613f89ca,NaN,NaN,"mtgo, paper",default,black,NaN,NaN,NaN,...,675ce267-e99d-523e-8722-402debaa9717,NaN,NaN,normal,USD,2023-11-05,paper,19.99,cardkingdom,retail


In [5]:
df.columns

Index(['artist', 'artistIds', 'asciiName', 'attractionLights', 'availability',
       'boosterTypes', 'borderColor', 'cardParts', 'colorIdentity',
       'colorIndicator', 'colors', 'defense', 'duelDeck', 'edhrecRank',
       'edhrecSaltiness', 'faceConvertedManaCost', 'faceFlavorName',
       'faceManaValue', 'faceName', 'finishes', 'flavorName', 'flavorText',
       'frameEffects', 'frameVersion', 'hand', 'hasAlternativeDeckLimit',
       'hasContentWarning', 'hasFoil', 'hasNonFoil', 'isAlternative',
       'isFullArt', 'isFunny', 'isOnlineOnly', 'isOversized', 'isPromo',
       'isRebalanced', 'isReprint', 'isReserved', 'isStarter',
       'isStorySpotlight', 'isTextless', 'isTimeshifted', 'keywords',
       'language', 'layout', 'leadershipSkills', 'life', 'loyalty', 'manaCost',
       'manaValue', 'name', 'number', 'originalPrintings',
       'originalReleaseDate', 'originalText', 'originalType', 'otherFaceIds',
       'power', 'printings', 'promoTypes', 'rarity', 'rebalancedPrint

In [6]:
unwanted_list = ('artist', 'artistIds', 'asciiName', 'attractionLights', 'availability',
       'boosterTypes', 'borderColor', 'cardParts', 'colorIdentity',
       'colorIndicator', 'colors', 'defense', 'duelDeck', 'manaCost',
       'faceConvertedManaCost', 'faceFlavorName', 'edhrecRank',	'edhrecSaltiness',
       'faceManaValue', 'faceName', 'finishes', 'flavorName', 'flavorText',
       'frameEffects', 'frameVersion', 'hand', 'hasAlternativeDeckLimit',
       'hasContentWarning', 'hasFoil', 'hasNonFoil', 'isAlternative',
       'isFullArt', 'isFunny', 'isOnlineOnly', 'isOversized', 'isPromo',
       'isRebalanced', 'isReprint', 'isReserved', 'isStarter',
       'isStorySpotlight', 'isTextless', 'isTimeshifted', 'keywords',
       'language', 'layout', 'leadershipSkills', 'life',
       'number', 'originalPrintings',
       'originalReleaseDate', 'originalText', 'otherFaceIds',
       'printings', 'promoTypes', 'rebalancedPrintings',
       'relatedCards', 'securityStamp', 'setCode', 'side', 'signature',
       'sourceProducts', 'subsets', 'subtypes', 'supertypes', 'type', 'types', 'uuid', 'variations', 'watermark',
       'cardFinish', 'currency', 'date', 'gameAvailability',
       'priceProvider', 'providerListing')

In [7]:
df_wanted = df.drop(columns=[col for col in df if col in unwanted_list])

df_wanted

,loyalty,manaValue,name,originalType,power,rarity,text,toughness,price
0,NaN,2.0,Angelic Page,Creature - Spirit,1,common,Flying\n{T}: Target attacking or blocking crea...,1,0.09
1,NaN,2.0,Angelic Page,Creature - Spirit,1,common,Flying\n{T}: Target attacking or blocking crea...,1,0.01
2,NaN,2.0,Angelic Page,Creature - Spirit,1,common,Flying\n{T}: Target attacking or blocking crea...,1,0.02
3,NaN,2.0,Angelic Page,Creature - Spirit,1,common,Flying\n{T}: Target attacking or blocking crea...,1,4.15
4,NaN,2.0,Angelic Page,Creature - Spirit,1,common,Flying\n{T}: Target attacking or blocking crea...,1,0.12
...,...,...,...,...,...,...,...,...,...
47366,NaN,0.0,Winding Canyons,Land,NaN,rare,"{T}: Add {C}.\n{2}, {T}: You may cast creature...",NaN,18.20
47367,NaN,0.0,Winding Canyons,Land,NaN,rare,"{T}: Add {C}.\n{2}, {T}: You may cast creature...",NaN,14.07
47368,NaN,0.0,Winding Canyons,Land,NaN,rare,"{T}: Add {C}.\n{2}, {T}: You may cast creature...",NaN,11.00
47369,NaN,0.0,Winding Canyons,Land,NaN,rare,"{T}: Add {C}.\n{2}, {T}: You may cast creature...",NaN,19.99


In [8]:
df_wanted.drop_duplicates('name', inplace = True)

In [9]:
df_wanted.dropna(subset='originalType',inplace=True)

In [10]:
df_wanted.fillna(-1,inplace=True)

C:\Users\drodm\AppData\Local\Temp\ipykernel_9832\444401854.py:1: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [11]:
df_wanted['raridade'] = df_wanted['rarity'].astype('category').cat.codes

In [12]:
df_wanted['classified_text'] = df_wanted['price'].astype('category').cat.codes

In [13]:
df_wanted

,loyalty,manaValue,name,originalType,power,rarity,text,toughness,price,raridade,classified_text
0,-1,2.0,Angelic Page,Creature - Spirit,1,common,Flying\n{T}: Target attacking or blocking crea...,1,0.09,0,7
11,-1,5.0,Ardent Militia,Creature - Soldier,2,uncommon,Vigilance,5,0.09,2,7
21,-1,2.0,Blessed Reversal,Instant,-1,rare,You gain 3 life for each creature attacking you.,-1,0.02,1,0
32,-1,4.0,Breath of Life,Sorcery,-1,uncommon,Return target creature card from your graveyar...,-1,0.04,2,2
43,-1,4.0,Castle,Enchantment,-1,uncommon,Untapped creatures you control get +0/+2.,-1,0.04,2,2
...,...,...,...,...,...,...,...,...,...,...,...
47331,-1,8.0,Xanthic Statue,Artifact,-1,rare,"{5}: Until end of turn, Xanthic Statue becomes...",-1,0.02,1,0
47339,-1,0.0,Gemstone Mine,Land,-1,uncommon,Gemstone Mine enters the battlefield with thre...,-1,2.76,2,154
47347,-1,0.0,Lotus Vale,Land,-1,rare,"If Lotus Vale would enter the battlefield, sac...",-1,0.42,1,39
47355,-1,0.0,Scorched Ruins,Land,-1,rare,"If Scorched Ruins would enter the battlefield,...",-1,0.23,1,21


In [14]:
df_wanted['classified_text'].min()

0

In [15]:
df_name = df_wanted['name']
df_wanted.drop(columns=['name'],inplace=True)
df_wanted.set_index(df_name, inplace=True)

In [16]:
df_wanted2 = df_wanted.drop(columns='loyalty')
notributes = ['power','toughness']
df_wanted3 = df_wanted2.drop(columns= [col for col in df_wanted2 if col in notributes])
df_wanted4 = df_wanted3.drop(columns='manaValue')
df_wanted5 = df_wanted.drop(columns= [col for col in df_wanted2 if col in notributes])

In [17]:
df_art = df_wanted2.loc[df_wanted['originalType'].str.contains('Artifact')]
df_mag = df_wanted3.loc[df_wanted['originalType'].str.contains('Sorcery')]
df_inst = df_wanted3.loc[df_wanted['originalType'].str.contains('Instant')]
df_enc = df_wanted3.loc[df_wanted['originalType'].str.contains('Enchantment')]
df_terr = df_wanted4.loc[df_wanted['originalType'].str.contains('Land')]
df_crea = df_wanted2.loc[df_wanted['originalType'].str.contains('Creature')]
df_planes = df_wanted5.loc[df_wanted['originalType'].str.contains('Planeswalker')]

In [18]:
df_art

,manaValue,originalType,power,rarity,text,toughness,price,raridade,classified_text
name,,,,,,,,,
Steal Artifact,4.0,Enchant Artifact,-1,uncommon,Enchant artifact\nYou control enchanted artifact.,-1,0.04,2,2
Aladdin's Ring,8.0,Artifact,-1,rare,"{8}, {T}: Aladdin's Ring deals 4 damage to any...",-1,0.02,1,0
Beast of Burden,6.0,Artifact Creature,*,rare,Beast of Burden's power and toughness are each...,*,0.02,1,0
Caltrops,3.0,Artifact,-1,uncommon,"Whenever a creature attacks, Caltrops deals 1 ...",-1,0.09,2,7
Charcoal Diamond,2.0,Artifact,-1,uncommon,Charcoal Diamond enters the battlefield tapped...,-1,0.06,2,4
...,...,...,...,...,...,...,...,...,...
Thran Forge,3.0,Artifact,-1,uncommon,"{2}: Until end of turn, target nonartifact cre...",-1,0.04,2,2
Thran Tome,4.0,Artifact,-1,rare,"{5}, {T}: Reveal the top three cards of your l...",-1,0.03,1,1
Touchstone,2.0,Artifact,-1,uncommon,{T}: Tap target artifact you don't control.,-1,0.06,2,4


In [19]:
df_crea

,manaValue,originalType,power,rarity,text,toughness,price,raridade,classified_text
name,,,,,,,,,
Angelic Page,2.0,Creature - Spirit,1,common,Flying\n{T}: Target attacking or blocking crea...,1,0.09,0,7
Ardent Militia,5.0,Creature - Soldier,2,uncommon,Vigilance,5,0.09,2,7
Cloudchaser Eagle,4.0,Creature - Bird,2,common,Flying\nWhen Cloudchaser Eagle enters the batt...,2,0.06,0,4
Crossbow Infantry,2.0,Creature - Soldier,1,common,{T}: Crossbow Infantry deals 1 damage to targe...,1,0.09,0,7
Eager Cadet,1.0,Creature - Soldier,1,common,-1,1,0.09,0,7
...,...,...,...,...,...,...,...,...,...
Nature's Kiss,2.0,Enchant Creature,-1,common,"Enchant creature\n{1}, Exile the top card of y...",-1,0.09,0,7
Jangling Automaton,3.0,Artifact Creature,3,common,"Whenever Jangling Automaton attacks, untap all...",2,0.09,0,7
Serrated Biskelion,3.0,Artifact Creature,2,uncommon,{T}: Put a -1/-1 counter on Serrated Biskelion...,2,0.04,2,2


In [20]:
df_crea.loc[df_crea['classified_text'].idxmax()]

manaValue                                                        1.0
originalType                                       Artifact Creature
power                                                             12
rarity                                                          rare
text               Trample\nWhen Phyrexian Dreadnought enters the...
toughness                                                         12
price                                                           8.25
raridade                                                           1
classified_text                                                  200
Name: Phyrexian Dreadnought, dtype: object

In [21]:
df_crea['originalType'].values

array(['Creature - Spirit', 'Creature - Soldier', 'Creature - Bird', ...,
       'Artifact Creature', 'Artifact Creature', 'Artifact Creature'],
      dtype=object)

In [22]:
to_drop2 = ['*','1+*','2+*','-1','+1','+3','3.5','?','∞','2.5','1.5','99','.5','*²','+4','+0','+2',-1]
for k in range(len(to_drop2)):
  df_crea = df_crea[df_crea.power != to_drop2[k]]

In [23]:
to_drop2 = ['*','1+*','+1','3.5','?','1.5','0',-1]
for k in range(len(to_drop2)):
  df_crea = df_crea[df_crea.toughness != to_drop2[k]]

In [24]:
df_crea['power'] = df_crea['power'].astype(float)
df_crea['toughness'] =df_crea['toughness'].astype(float)

In [25]:
#df_crea.corr()

In [62]:
df_name = pd.read_csv("Names.csv")    
df_name.drop(columns="Unnamed: 0", inplace=True)
k = df_name.iloc[np.where(df_name['name'].str.contains('//'))]
k = k.index.to_list()
for ki in k:
    nme = df_name.name[ki:ki+1].to_list()
    for n in nme:
        nme = n.split('//', 1)
    df_name[ki:ki+1] = nme[0]


,name
0,Ancestor's Chosen
1,Angel of Mercy
2,Angelic Blessing
3,Angelic Chorus
4,Angelic Wall
...,...
22123,Eternity Vessel
22124,Hedron Scrabbler
22125,Khalni Gem
22126,"Magosi, the Waterveil"


In [4]:
df_name.loc[df_name.name == "Prowl, Stoic Strategist // Prowl, Pursuit Vehicle"]

,name
4785,"Prowl, Stoic Strategist // Prowl, Pursuit Vehicle"


In [5]:
teste = df_name.name[4785:4799].to_list()#4799
correctones = []
for t in teste:
    result = t.split('//', 1)
    print(result[0])
    correctones.append(f'{result[0]}')

Prowl, Stoic Strategist 
Ratchet, Field Medic 
Jetfire, Ingenious Scientist 
Blitzwing, Cruel Tormentor 
Starscream, Power Hungry 
Slicer, Hired Muscle 
Arcee, Sharpshooter 
Cyclonus, the Saboteur 
Flamewar, Brash Veteran 
Goldbug, Humanity's Ally 
Megatron, Tyrant 
Optimus Prime, Hero 
Soundwave, Sonic Spy 
Ultra Magnus, Tactician 


In [6]:
df_corr = pd.DataFrame({'name': correctones})

In [7]:
df_name.iloc[4785:4799] = df_corr.values

In [85]:
df_name.iloc[21816:21817] = "Intrepid Trufflesnout"

In [86]:
df_name.loc[21815:]

,name
21815,Intrepid Trufflesnout
21816,Intrepid Trufflesnout
21817,Provisions Merchant
21818,Wildwood Mentor
21819,Spreading Seas
...,...
22123,Eternity Vessel
22124,Hedron Scrabbler
22125,Khalni Gem
22126,"Magosi, the Waterveil"


In [29]:
k = df_name.iloc[np.where(df_name['name'].str.contains('//'))]
k = k.index.to_list()
for ki in k:
    nme = df_name.name[ki:ki+1].to_list()
    for n in nme:
        nme = n.split('//', 1)
    df_name[ki:ki+1] = nme[0]

In [92]:
f_df = pd.read_csv('ScrapingCards.csv')

In [88]:
len(valores)

21695

In [90]:
def testando():
  driver = webdriver.Chrome()
  cards = df_name.name[21816:].to_list()
  tamanho = len(cards)
  for card in tqdm(cards, desc= "Loop Ligamagic"):
    donecards.append(f'{card}')
    driver.get("https://www.ligamagic.com.br/?view=cards/card&card="+f'{card}')
    driver.set_window_size(1054, 800)
    try:
      to_click = driver.find_elements(By.CLASS_NAME, "mtg-name-aux")
      for t in to_click:
        if(t.get_attribute('innerText') == f'{card}'):
          tref=t.find_element(By.TAG_NAME, 'a')
          tref=tref.get_attribute('href')
          driver.get(tref)
          break
    except:
      pass

    try:
      valor = driver.find_element(By.CLASS_NAME, "col-prc-menor")
      valor = valor.get_attribute('innerText')
    except:
      valor = "R$ 0,00"
    valores.append(f'{valor}')
    cmc = driver.find_elements(By.CLASS_NAME, "col-lg-6")
    for e in cmc:
      lista = e.find_elements(By.TAG_NAME, "a")
    
    try:
      cmc = lista.pop()
      cmc = cmc.text
    except:
      cmc = '0'
    cmcs.append(f'{cmc}')

    raridade = driver.find_element(By.ID, "ed-raridade")
    raridade = raridade.find_element(By.TAG_NAME, "a")
    raridade = raridade.get_attribute('innerText')
    raridades.append(f'{raridade}')
    
    driver.get("https://scryfall.com/")
    driver.set_window_size(1054, 800)
    for c in card:
      driver.find_element(By.ID, "q").send_keys(f"{c}")
    
    driver.find_element(By.ID, "q").send_keys(Keys.ENTER)

    tohover = driver.find_elements(By.CLASS_NAME , "card-grid-item-card")
    for t in tohover:
      if(t.get_attribute('innerText') == f'{card}'):
        tref=t.get_attribute('href')
        driver.get(tref)
        break
      
    try:
      tp = driver.find_element(By.CLASS_NAME, "card-text-type-line")
      tp = tp.get_attribute('innerText')
    except:
      driver.find_element(By.CLASS_NAME, "card-grid-item-card").click()
      tp = driver.find_element(By.CLASS_NAME, "card-text-type-line")
      tp = tp.get_attribute('innerText')
    
    tipos.append(f'{tp}')
    try:
      texto = driver.find_element(By.CLASS_NAME, "card-text-oracle")
      texto = texto.find_elements(By.TAG_NAME, "p")
      textoa = []
      for txt in texto:
        tx = txt.get_attribute('innerText')
        textoa.append(tx)
      separator = ''
      texto = separator.join(textoa)
    except:
      texto = "null"
    
    textos.append(f'{texto}')

    try:
        pr = driver.find_element(By.CLASS_NAME, "card-text-stats")
        pr = pr.get_attribute('innerText')
        prlist = []
        
        prlist = pr.split('/')
        if(prlist[1]):
          ps.append(f'{prlist[0]}')
          rs.append(f'{prlist[1]}')
          loyals.append(f'{'NaN'}')
        else:
          loyals.append(f'{prlist[0]}')
    except:
        pr = "NaN/NaN"
        prlist = pr.split('/')

        loyals.append(f'{'NaN'}')
        ps.append(f'{prlist[0]}')
        rs.append(f'{prlist[1]}')
    data = pd.DataFrame({
      'name': donecards,
      'tipo': tipos,
      'cmc': cmcs,
      'power': ps,
      'resistence': rs,
      'lealdade': loyals,
      'texto': textos,
      'raridades': raridades,
      'preços': valores})
    data['preços'].str.strip("R$")
    data['preços'].replace(',','.', regex=True, inplace=True)
    data.to_csv(r'ScrapingCards.csv', mode="w")

  driver.quit()
  

In [91]:
print(len(cmcs))
print(len(valores))
print(len(rs))
print(len(tipos))
print(len(textos))
print(len(ps))
print(len(loyals))
print(len(raridades))

21695
21695
21695
21695
21695
21695
21695
21695


In [93]:
testando()

Loop Ligamagic: 100%|██████████| 312/312 [18:51<00:00,  3.63s/it]


# Criando o modelo de classificação de textos

In [ ]:
# data_texts = df_crea['text'].astype(str).to_list()
# data_labels = df_crea['classified_text'].to_list()
# train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size = 0.8, random_state = 0 )

In [ ]:
# train_texts

In [ ]:
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
# train_encodings = tokenizer(train_texts, truncation = True, padding = True  )

# val_encodings = tokenizer(val_texts, truncation = True, padding = True )

In [ ]:
# train_encodings

In [ ]:
# train_dataset = tf.data.Dataset.from_tensor_slices((
#      dict(train_encodings),
#      train_labels
#  ))


# val_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(val_encodings),
#      val_labels
#  ))

In [ ]:
# model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

In [ ]:
# from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments


# training_args = TFTrainingArguments(
#     output_dir='/content/drive/Shareddrives/Ufla/Pattern_Recognition/results',
#     num_train_epochs=20,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=64,
#     warmup_steps=50000,
#     weight_decay=1e-5,
#     logging_dir='/content/drive/Shareddrives/Ufla/Pattern_Recognition/logs',
#     eval_steps=1000
# )

# with training_args.strategy.scope():
#     trainer_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 20)


# trainer = TFTrainer(
#     model=trainer_model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
# )

In [ ]:
# trainer.train()

In [ ]:
# trainer.evaluate()

In [ ]:
save_directory = "/content/drive/Shareddrives/Ufla/Pattern_Recognition/saved_models"

# model.save_pretrained(save_directory)

# tokenizer.save_pretrained(save_directory)

In [ ]:
tokenizer_fine_tuned = DistilBertTokenizer.from_pretrained(save_directory)

model_fine_tuned = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

OSError: Incorrect path_or_model_id: '/content/drive/Shareddrives/Ufla/Pattern_Recognition/saved_models'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

# Clusterização


In [ ]:
noatributes = ['originalType','text','price','rarity','pred_price']
features = df_crea.drop(columns= [col for col in df_crea if col in noatributes])
target = df_crea['price']

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=.8, random_state=None) # hold out 20% of the data for final testing

(2122, 5)

In [ ]:
label_encoder = LabelEncoder()

true_labels = label_encoder.fit_transform(target)

In [ ]:
label_encoder.classes_

n_clusters = len(label_encoder.classes_)

print(n_clusters)

In [ ]:
preprocessor = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        ("pca", PCA(n_components=5, random_state=42)),
    ]
)

In [ ]:
clusterer = Pipeline(
    [
        (
            "kmeans",
            KMeans(
                n_clusters=n_clusters,
                init="k-means++",
                n_init=50,
                max_iter=500,
                random_state=42,
            ),
         ),
    ]
 )

In [ ]:
pipe = Pipeline(
     [
         ("preprocessor", preprocessor),
         ("clusterer", clusterer)
     ]
   )

In [ ]:
pipe.fit(features)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('scaler', MinMaxScaler()),
                                 ('pca',
                                  PCA(n_components=5, random_state=42))])),
                ('clusterer',
                 Pipeline(steps=[('kmeans',
                                  KMeans(max_iter=500, n_clusters=539,
                                         n_init=50, random_state=42))]))])

In [ ]:
preprocessed_data = pipe["preprocessor"].transform(features)

predicted_labels = pipe["clusterer"]["kmeans"].labels_

silhouette_score(preprocessed_data, predicted_labels)

0.7258360548512859

In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score
adjusted_rand_score(true_labels, predicted_labels)

0.013069682411475759

In [ ]:
dfcluster = pd.DataFrame(
    pipe["preprocessor"].transform(features),
    columns=features.columns,
)

dfcluster["predicted_cluster"] = pipe["clusterer"]["kmeans"].labels_
dfcluster["true_label"] = label_encoder.inverse_transform(true_labels)

plt.style.use("fivethirtyeight")
plt.figure(figsize=(8, 8))

scat = sns.scatterplot(
    data=dfcluster[['classified_text','predicted_cluster',"true_label"]],
    x = "classified_text",
    y = "predicted_cluster",
    s=50,
    hue="predicted_cluster",
    style="true_label",
    palette="Set2",
)

scat.set_title(
     "Clustering results Magic/price"
)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

plt.show()

In [ ]:
dfcluster

,manaValue,power,toughness,raridade,classified_text,predicted_cluster,true_label
0,0.564800,0.180737,-0.045401,-0.075880,-0.063937,148,0.04
1,0.558836,0.037315,-0.031574,-0.041155,-0.029975,27,0.04
2,-0.446936,-0.077428,-0.017868,0.160701,-0.071230,157,0.04
3,-0.444533,-0.030126,-0.024047,-0.045148,-0.038166,33,0.04
4,0.554679,-0.069117,-0.022137,-0.041493,-0.031888,53,0.03
...,...,...,...,...,...,...,...
10607,-0.444534,-0.030216,-0.024973,-0.045143,-0.038175,33,0.03
10608,0.056879,-0.012677,-0.027944,-0.078380,-0.070906,170,0.02
10609,-0.440377,0.076217,-0.034410,-0.044805,-0.036263,8,0.04
10610,-0.450497,-0.173638,-0.011145,-0.010418,-0.004213,0,0.03


In [ ]:
dfcluster['true_label'].loc[dfcluster['predicted_cluster'] == int(pipe.predict(test_input))]

369     0.02
969     0.11
2162    0.95
2494    0.36
2726    0.26
3947    0.02
4449    0.73
6604    0.10
6672    1.21
6990    0.47
7626    0.06
7648    0.02
8065    0.42
8150    0.02
8646    0.02
9791    0.02
Name: true_label, dtype: float64

# Previsão

In [ ]:
test_text = "Flying, vigilance At the beginning of your upkeep, investigate once for each opponent who has more cards in hand than you. "

#test_text

In [ ]:
predict_input = tokenizer_fine_tuned.encode(
    test_text,
    truncation = True,
    padding = True,
    return_tensors = 'tf'
)

output = model_fine_tuned(predict_input)[0]

prediction_value = tf.argmax(output, axis = 1).numpy()[0]

prediction_value

0

In [ ]:
tokenizer_fine_tuned_pt = DistilBertTokenizer.from_pretrained(save_directory)


model_fine_tuned_pt = DistilBertForSequenceClassification.from_pretrained(save_directory, from_tf = True )

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


In [ ]:
predict_input_pt = tokenizer_fine_tuned_pt.encode(test_text, truncation = True, padding = True, return_tensors = 'pt' )

ouput_pt = model_fine_tuned_pt(predict_input_pt)

prediction_value_pt = torch.argmax(ouput_pt[0], dim = 1 ).item()

prediction_value_pt

0

In [ ]:
# alterar valores conforme atributos da carta a ser testada e o valor de saída de classificação do texto
valores = [[3, 2, 4, 2, 0]]

test_input = pd.DataFrame(valores, columns=['manaValue', 'power', 'toughness', 'raridade', 'classified_text'])

In [ ]:
dfcluster['true_label'].loc[dfcluster['predicted_cluster'] == int(pipe.predict(test_input))].mean()

0.299375

# Deploy versão 2.0


In [ ]:
# dfs = {'creatures': df_crea , 'artefatos' : df_art }
# for key, df in dfs.items():
#   data_texts = df['text'].astype(str).to_list()
#   data_labels = df['classified_text'].to_list()
#   train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size = 0.8, random_state = 0 )

#   train_encodings = tokenizer(train_texts, truncation = True, padding = True  )
#   val_encodings = tokenizer(val_texts, truncation = True, padding = True )

#   train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(train_encodings),
#     train_labels
#     ))


#   val_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(val_encodings),
#     val_labels
#     ))
#   training_args = TFTrainingArguments(
#     output_dir=f'/content/drive/Shareddrives/Ufla/Pattern_Recognition/results/{key}',
#     num_train_epochs=10,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=64,
#     warmup_steps=10000,
#     weight_decay=1e-5,
#     logging_dir=f'/content/drive/Shareddrives/Ufla/Pattern_Recognition/logs/{key}',
#     eval_steps=100
#     )

#   with training_args.strategy.scope():
#       trainer_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 20 )


#   trainer = TFTrainer(
#       model=trainer_model,
#       args=training_args,
#       train_dataset=train_dataset,
#       eval_dataset=val_dataset,
#   )

#   trainer.train()
#   trainer.evaluate()